In [1]:
import os
import sys
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np
import itertools
from nltk import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, \
StackingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay, recall_score,\
    accuracy_score, precision_score, f1_score, classification_report
import warnings
import xgboost

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('fake reviews dataset.csv')
df['target'] = np.where(df['label'] == 'CG', 1, 0)

In [3]:
#tokenizing the text data in the 'text_' column of df
def tokenizer(x):
    
    corpus = [word_tokenize(doc) for doc in x]

# getting common stop words in english that we'll remove during tokenization/text normalization
    stop_words = stopwords.words('english')
    corpus_no_stopwords = []
    for words in corpus:
        docs = [x.lower() for x in words if ((x.isalpha()) & (x not in stop_words))]
        corpus_no_stopwords.append(docs)
    return corpus_no_stopwords


def lemmatizer(corpus, as_string=True):
    lem = WordNetLemmatizer()
    
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None
    lemmatized_corpus = []
    for sentence in corpus:
        pos_tags = pos_tag(sentence)
        lemmatized_sentence = []
        for word, tag in pos_tags:
            pos = pos_tagger(tag)
            if pos is not None:
                lemmatized_word = lem.lemmatize(word, pos)
            else:
                lemmatized_word = lem.lemmatize(word)
            lemmatized_sentence.append(lemmatized_word)
        lemmatized_corpus.append(lemmatized_sentence)
    if as_string:
        lemmatized_corpus  = [' '.join(x) for x in lemmatized_corpus]
    return lemmatized_corpus
        

In [4]:
X_train, X_test, y_train,y_test = train_test_split(df['text_'],df['target'], test_size=0.3,random_state=42)

In [13]:
X_train_preprocessed = lemmatizer(tokenizer(X_train))
X_test_preprocessed = lemmatizer(tokenizer(X_test))

In [18]:
vectorizer = CountVectorizer(min_df=0.05,max_df=0.95)

X_train_vec = vectorizer.fit_transform(X_train_preprocessed)
X_test_vec = vectorizer.transform(X_test_preprocessed)

## Here we appply count vectorizer

In [6]:
random_forest_hp = RandomForestClassifier(max_depth=5,min_samples_leaf=2,n_estimators=100, random_state=42)

In [7]:
logistic_regression_best_hp = LogisticRegression(C=1,penalty='l1',solver='liblinear', random_state=42)

In [8]:
ada_booster_best_hp = AdaBoostClassifier(learning_rate=0.5,n_estimators=200)

In [9]:
gradient_booster_classifier_best_hp = GradientBoostingClassifier(learning_rate=0.5, max_depth=4,n_estimators=200)

In [10]:
xgbooster_best_hp = xgboost.XGBClassifier(random_state=42, objective='binary:logistic',gamma=0.2, learning_rate=0.1, max_depth=7, n_estimators=200, reg_lambda=1.5, subsample=0.9)

In [11]:
#CREATE PARAM GRID FOR NAIVE BAYES
#GRID SEARCH NAIVE BAYES
# SAVE BEST MODEL
#START BAGGING

In [12]:
steps = [('countv',CountVectorizer(min_df=0.05,max_df=0.95)),('bagg',BaggingClassifier(estimator=xgbooster_best_hp, n_estimators=100))]
bagging_pipeline = Pipeline(steps)
bagging_pipeline.fit(X_train_preprocessed, y_train)
crossval_xgb_bagging = np.mean(cross_val_score(bagging_pipeline, X_train_preprocessed,y_train,scoring='accuracy',cv=5))
crossval_xgb_bagging

AttributeError: 'list' object has no attribute 'lower'

In [ ]:
X_train_preprocessed

In [19]:
bagging = BaggingClassifier(estimator=xgboost.XGBClassifier(random_state=42, objective='binary:logistic',gamma=0.2, learning_rate=0.1, max_depth=7, n_estimators=200, reg_lambda=1.5, subsample=0.9), n_estimators=100)
bagging.fit(X_train_vec,y_train)

BaggingClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0.2, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0.1, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=7,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=200, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=42, ...),
                  n_estimators=100)

In [20]:
crossval_xgb_bagging = np.mean(cross_val_score(bagging, X_train_vec,y_train,scoring='accuracy',cv=5))
crossval_xgb_bagging

0.7782489078144867

In [21]:
X_train_vec

<28302x91 sparse matrix of type '<class 'numpy.int64'>'
	with 266047 stored elements in Compressed Sparse Row format>